In [ ]:
%pip install fiona
%pip install rtree==1.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for rtree: filename=rtree-1.0.0-cp312-cp312-linux_x86_64.whl size=24160 sha256=872a236bedd42f96ea0a19897f9c23470aea99f1d48bae0bd993b950f0fc7a48
  Stored in directory: /root/.cache/pip/wheels/dd/1b/85/3d3f1291a6b2d664f67de54bba659f4159391f4af09e60d47d
Successfully built rtree


In [ ]:
import geopandas as gpd
from shapely.ops import linemerge
from shapely.geometry import Point, LineString, MultiLineString
import pandas as pd
import geopandas as gpd
import fiona
from shapely.geometry import shape, Point, LineString, MultiLineString
from shapely.ops import linemerge, transform as shp_transform
from shapely.strtree import STRtree
import pyproj
import numpy as np
import folium

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
rios   = gpd.read_file('/content/drive/MyDrive/Mestrado/v_hidrografia_grass_2k.gpkg').to_crs(epsg=31983)
pontos = gpd.read_file('/content/drive/MyDrive/Mestrado/v_pontos_micro_2_BR.gpkg').to_crs(epsg=31983)


In [ ]:
# --- explode multipoint para ter 1 ponto por linha ---
pontos = pontos.explode(index_parts=True).reset_index(drop=True)

# identifica a coluna de ID do ponto
ID_COL = 'ponto_de_coleta' if 'ponto_de_coleta' in pontos.columns else None

def to_linestring(geom):
    """Converte MultiLineString -> LineString"""
    if geom is None:
        return None
    if geom.geom_type == "LineString":
        return geom
    if geom.geom_type == "MultiLineString":
        merged = linemerge(geom)
        if merged.geom_type == "LineString":
            return merged
        else:  # MultiLineString ainda
            return max(merged.geoms, key=lambda g: g.length)
    return None

# índice espacial para buscar rio mais próximo
sidx = rios.sindex

resultados = []

for i, geom_point in enumerate(pontos.geometry): # Iterate directly over geometries
    prow = pontos.iloc[i] # Access the corresponding row data
    if geom_point is None:
        continue
    if geom_point.geom_type != "Point":
        geom_point = geom_point.centroid

    # acha candidato mais próximo
    idxs = list(sidx.nearest(geom_point))
    if not idxs:
        continue

    rio_row = rios.iloc[idxs[1]]
    linha = to_linestring(rio_row.geometry.item()) # Access the geometry object from the Series
    if linha is None:
        continue

    proj = linha.interpolate(linha.project(geom_point))
    L = linha.length
    d = linha.project(proj)

    resultados.append({
        "id_ponto": prow.get(ID_COL, f"ponto_{i}"),
        "nm_cursodagua": rio_row.get('nm_cursodagua', None),
        "dist_montante_m": float(d),
        "dist_jusante_m": float(L - d),
        "comprimento_trecho_m": float(L),
        "x_proj": float(proj.x),
        "y_proj": float(proj.y),
    })

df_res = pd.DataFrame(resultados)
df_res.to_csv('/content/resultado.csv', sep=';', decimal=',', index=False)
df_res.head()

,id_ponto,nm_cursodagua,dist_montante_m,dist_jusante_m,comprimento_trecho_m,x_proj,y_proj
0,MBR-02,None,7.532647,35.450611,42.983259,-416342.992871,7.251013e+06
1,MBR-04,None,2199.202604,214.565508,2413.768112,-415963.679181,7.227910e+06
2,MBR-11,None,3201.652660,178.461192,3380.113853,-416941.474967,7.198114e+06
3,MBR-13,None,367.286709,28.295271,395.581981,-401435.552981,7.196364e+06
4,MBR-15,None,2730.716204,5737.208611,8467.924815,-427358.256351,7.174936e+06


Mapa


In [ ]:
# !pip install -q shapely==2.* pyproj fiona folium

import fiona
from shapely.geometry import shape, LineString, MultiLineString
from shapely.ops import linemerge, transform as shp_transform
from shapely.strtree import STRtree
import pyproj, numpy as np, folium, pathlib, os
from google.colab import files   # <- importa utilitário de download no Colab

# ============================
# CAMINHOS
# ============================
PATH_RIOS   = '/content/drive/MyDrive/Mestrado/v_hidrografia_grass_2k.gpkg'
PATH_PONTOS = '/content/drive/MyDrive/Mestrado/v_pontos_micro_2_BR.gpkg'
OUTPUT_MAP  = '/content/mapa_dist_proj.html'   # salva no /content

CRS_METERS_EPSG = 31983
CRS_WGS84_EPSG  = 4326

# ============================
# FUNÇÕES AUX
# ============================
def read_geoms_with_props(path):
    items = []
    with fiona.open(path) as src:
        crs = src.crs_wkt if src.crs_wkt else src.crs
        for feat in src:
            g = shape(feat['geometry']) if feat['geometry'] else None
            items.append((g, feat['properties']))
    return items, crs

def transformer(src_crs, dst_epsg):
    src = pyproj.CRS.from_user_input(src_crs) if src_crs else None
    dst = pyproj.CRS.from_epsg(dst_epsg)
    return pyproj.Transformer.from_crs(src or dst, dst, always_xy=True)

def reproj(g, tf): return shp_transform(tf.transform, g) if g is not None else None

# ============================
# CARREGAR
# ============================
(rivers_raw, crs_rios) = read_geoms_with_props(PATH_RIOS)
(points_raw, crs_pts)  = read_geoms_with_props(PATH_PONTOS)

to_m_rios = transformer(crs_rios, CRS_METERS_EPSG)
to_m_pts  = transformer(crs_pts,  CRS_METERS_EPSG)
to_wgs84_from_m = pyproj.Transformer.from_crs(CRS_METERS_EPSG, CRS_WGS84_EPSG, always_xy=True)

# rios
river_lines, river_props = [], []
for g, props in ((reproj(g, to_m_rios), p) for g, p in rivers_raw):
    if g is None: continue
    if isinstance(g, (LineString, MultiLineString)):
        if g.geom_type == 'MultiLineString':
            merged = linemerge(g)
            ln = merged if merged.geom_type == 'LineString' else (
                max(merged.geoms, key=lambda gg: gg.length) if len(merged.geoms) else None
            )
        else: ln = g
        if ln is not None and ln.length > 0:
            river_lines.append(ln); river_props.append(props)

# pontos
points_m = []
for g, props in ((reproj(g, to_m_pts), p) for g, p in points_raw):
    if g and g.geom_type != 'Point': g = g.centroid
    points_m.append((g, props))

# ============================
# PROJEÇÕES
# ============================
tree = STRtree(river_lines)
geom_to_idx = {id(geom): i for i, geom in enumerate(river_lines)}
ID_COL = 'ponto_de_coleta' if points_raw and ('ponto_de_coleta' in (points_raw[0][1] or {})) else None

rows, proj_pts, links = [], [], []
for i,(pt,props) in enumerate(points_m):
    if pt is None or pt.is_empty: continue
    ln = tree.nearest(pt); ridx = geom_to_idx.get(id(ln))
    if ridx is None: continue
    rprops = river_props[ridx]
    proj = ln.interpolate(ln.project(pt))
    L, d = float(ln.length), float(ln.project(proj))
    rows.append({
        'id_ponto': props.get(ID_COL) if (ID_COL and props) else f'ponto_{i}',
        'nm_cursodagua': rprops.get('nm_cursodagua') if rprops else None,
        'dist_montante_m': d, 'dist_jusante_m': L-d, 'comprimento_trecho_m': L
    })
    proj_pts.append(proj); links.append(LineString([pt, proj]))

def to_wgs84(g): return shp_transform(to_wgs84_from_m.transform, g)

# centro do mapa
pts_wgs = [to_wgs84(pt) for pt,_ in points_m if pt is not None and not pt.is_empty]
center = (float(np.mean([g.y for g in pts_wgs])), float(np.mean([g.x for g in pts_wgs]))) if pts_wgs else (-25.5,-54.6)

# ============================
# MAPA
# ============================
m = folium.Map(location=center, zoom_start=10, tiles="OpenStreetMap")

# rios
for ln in river_lines:
    g = to_wgs84(ln)
    if g.geom_type == 'LineString':
        folium.PolyLine([(y,x) for x,y in np.array(g.coords)], weight=2).add_to(m)
    else:
        for part in g.geoms:
            folium.PolyLine([(y,x) for x,y in np.array(part.coords)], weight=2).add_to(m)

# links
for seg in links:
    g = to_wgs84(seg)
    folium.PolyLine([(pt[1], pt[0]) for pt in np.array(g.coords)], color="red").add_to(m)

# pontos projetados
for r,proj_m in zip(rows, proj_pts):
    g = to_wgs84(proj_m)
    folium.CircleMarker((g.y,g.x),radius=4,color="blue",fill=True).add_child(
        folium.Popup(f"{r['id_ponto']}<br>{r['dist_montante_m']:.1f} m / {r['dist_jusante_m']:.1f} m")
    ).add_to(m)

# pontos originais
for (pt,props) in points_m:
    if pt is None or pt.is_empty: continue
    g = to_wgs84(pt)
    label = str(props.get(ID_COL,"ponto")) if props else "ponto"
    folium.Marker((g.y,g.x),tooltip=label).add_to(m)

m.save(OUTPUT_MAP)
print("Mapa salvo em:", OUTPUT_MAP)

# ============================
# DOWNLOAD AUTOMÁTICO
# ============================
files.download(OUTPUT_MAP)


Mapa salvo em: /content/mapa_dist_proj.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import geopandas as gpd
from shapely.ops import linemerge
from shapely.geometry import Point, LineString, MultiLineString
import pandas as pd
import geopandas as gpd
import fiona
from shapely.geometry import shape, Point, LineString, MultiLineString
from shapely.ops import linemerge, transform as shp_transform
from shapely.strtree import STRtree
import pyproj
import numpy as np
import folium
import networkx as nx


PATH_RIOS   = "/content/drive/MyDrive/Mestrado/v_hidrografia_grass_2k.gpkg"
PATH_PONTOS = "/content/drive/MyDrive/Mestrado/v_pontos_micro_2_BR.gpkg"
CRS_METERS  = 31983   # SIRGAS 2000 / UTM 22S
OUT_CSV     = "/content/distancia_ate_tronco_principal.csv" # Changed output path to /content
OUT_HTML    = "/content/mapa_dist_ate_tronco.html" # Changed output path to /content

# Se existir uma coluna de nome do curso d'água, usamos "Paran" para detectar o tronco
MAINSTEM_NAME_SUBSTR = "paran"
POINT_ID_CAND = ["ponto_de_coleta","id","nome","ponto","label"]

# ------------- escolher as camadas corretas dentro do GPKG -------------
def pick_layer_by_geom(path, wanted=("LineString","MultiLineString")):
    layers = fiona.listlayers(path)
    for lyr in layers:
        with fiona.open(path, layer=lyr) as src:
            gtype = src.schema["geometry"]
            if any(w.lower() in gtype.lower() for w in wanted):
                return lyr
    return layers[0]

lyr_rios = pick_layer_by_geom(PATH_RIOS, ("LineString","MultiLineString"))
lyr_pts  = pick_layer_by_geom(PATH_PONTOS, ("Point","MultiPoint"))

rios   = gpd.read_file(PATH_RIOS,  layer=lyr_rios)
pontos = gpd.read_file(PATH_PONTOS, layer=lyr_pts)

# ------------- garantir CRS em metros -------------
rios   = rios.to_crs(epsg=CRS_METERS)
pontos = pontos.to_crs(epsg=CRS_METERS)

# ------------- explode multipoint -------------
pontos = pontos.explode(index_parts=True).reset_index(drop=True)

# ------------- escolher coluna de ID -------------
ID_COL = next((c for c in POINT_ID_CAND if c in pontos.columns), None)
if ID_COL is None:
    ID_COL = "id_ponto"
    pontos[ID_COL] = [f"ponto_{i}" for i in range(len(pontos))]

# ------------- normalizar linhas (Multi -> LineString) -------------
def to_linestring(geom):
    if geom is None: return None
    if isinstance(geom, LineString):       return geom
    if isinstance(geom, MultiLineString):
        merged = linemerge(geom)
        if isinstance(merged, LineString): return merged
        # If linemerge still results in a MultiLineString, take the longest part
        if isinstance(merged, MultiLineString):
             return max(list(merged.geoms), key=lambda g: g.length, default=None)
        return merged # Should be a LineString or None
    return None

rios["geom_ls"] = rios.geometry.apply(to_linestring)
rios = rios.dropna(subset=["geom_ls"]).set_geometry("geom_ls", crs=rios.crs)

# ------------- detectar tronco principal -------------
name_col = next((c for c in ["nm_cursodagua","nome","name","nm_trecho","river_name"] if c in rios.columns), None)
if name_col:
    mask_main = rios[name_col].astype(str).str.lower().str.contains(MAINSTEM_NAME_SUBSTR, na=False)
else:
    # fallback: pega os 2% maiores comprimentos como proxy de tronco
    lens = rios.length
    thr = np.quantile(lens, 0.98)
    mask_main = lens >= thr

mainstem = rios[mask_main]
if mainstem.empty:
    # fallback adicional: pegue os 20 maiores trechos
    mainstem = rios.sort_values(rios.geometry.length, ascending=False).head(20)

# ------------- construir o grafo da drenagem -------------
G = nx.Graph()

def add_ls(ls: LineString):
    coords = list(ls.coords)
    for a, b in zip(coords[:-1], coords[1:]):
        da, db = (a[0], a[1]), (b[0], b[1])
        seg = LineString([da, db])
        G.add_edge(da, db, weight=seg.length, geometry=seg)

# Filter out any remaining non-LineString geometries before adding to graph
for ls in rios.geometry:
    if isinstance(ls, LineString) and not ls.is_empty:
        add_ls(ls)


# Nós-alvo = nós pertencentes ao tronco principal
target_nodes = set()
for ls in mainstem.geometry:
    if isinstance(ls, LineString) and not ls.is_empty:
      for a, b in zip(list(ls.coords)[:-1], list(ls.coords)[1:]):
          target_nodes.add((a[0], a[1])); target_nodes.add((b[0], b[1]))

# ------------- índice espacial p/ pegar linha mais próxima -------------
sidx = rios.sindex

def nearest_linestring(pt: Point):
    cand = list(sidx.nearest(pt))
    best_ls, best_d = None, 1e30
    for i in cand:
        ls_candidate = rios.geometry.iloc[i]
        # Ensure ls_candidate is a valid LineString before proceeding
        if isinstance(ls_candidate, LineString) and not ls_candidate.is_empty:
            d = pt.distance(ls_candidate)
            if d < best_d:
                best_d, best_ls = d, ls_candidate
    return best_ls

def shortest_to_mainstem(pt: Point):
    ls = nearest_linestring(pt)
    if ls is None: return None, None, None

    proj = ls.interpolate(ls.project(pt))
    tnode = (proj.x, proj.y)

    # ligar nó temporário aos dois extremos do segmento onde projetou
    coords = list(ls.coords)
    endA, endB = (coords[0][0], coords[0][1]), (coords[-1][0], coords[-1][1])

    # adiciona arestas temporárias
    segA = LineString([tnode, endA])
    segB = LineString([tnode, endB])
    # Add edges only if the nodes are valid in the graph (can be reached)
    if endA in G and endB in G:
        G.add_edge(tnode, endA, weight=segA.length, geometry=segA)
        G.add_edge(tnode, endB, weight=segB.length, geometry=segB)
    else:
        # If endpoints are not in the graph, the projection is likely problematic or on a disconnected segment
        if tnode in G: G.remove_node(tnode)
        return None, None, None


    # dijkstra até qualquer nó do tronco
    best_len, best_tgt, best_path = None, None, None
    # Check if tnode is in the graph before running dijkstra
    if tnode in G:
        lengths, paths = nx.multi_source_dijkstra(G, sources=[tnode], weight="weight")
        for tgt in target_nodes:
            if tgt in lengths:
                L = lengths[tgt]
                if (best_len is None) or (L < best_len):
                    best_len, best_tgt, best_path = L, tgt, paths[tgt]

    # limpar temporários
    if tnode in G and G.has_edge(tnode, endA):
        G.remove_edge(tnode, endA)
    if tnode in G and G.has_edge(tnode, endB):
        G.remove_edge(tnode, endB)
    if tnode in G and G.degree[tnode] == 0: # Only remove if no other edges are connected
         G.remove_node(tnode)

    if best_len is None:
        return None, None, None

    # reconstruir geometria do caminho
    if best_path and len(best_path) >= 2:
        path_ls = LineString([(x, y) for (x, y) in best_path])
    else:
        path_ls = None

    return float(best_len), best_tgt, path_ls

# ------------- rodar para todos os pontos -------------
rows = []
paths = []
for idx, r in pontos.iterrows(): # Added idx for potential debugging
    # Ensure the point geometry is valid before processing
    if r.geometry is not None and not r.geometry.is_empty and r.geometry.geom_type == 'Point':
        d, tgt, pls = shortest_to_mainstem(r.geometry)
        if d is not None: # Check if distance is not None before appending
            rows.append({"id_ponto": r[ID_COL], "dist_m_ate_tronco": d})
            if pls is not None: # Check if path geometry is not None before appending
                paths.append({"id_ponto": r[ID_COL], "geometry": pls})
        else:
            print(f"Warning: Could not find shortest path for point {r.get(ID_COL, idx)}") # Added print statement for debugging


df_out = pd.DataFrame(rows) # Create DataFrame from rows
if not df_out.empty: # Check if DataFrame is not empty before sorting
    df_out = df_out.sort_values("dist_m_ate_tronco")

df_out.to_csv(OUT_CSV, index=False)

# ------------- mapa Folium (rios azuis, tronco grosso, caminho vermelho) -------------
if len(pontos) > 0:
    # Find a valid point geometry for centering the map
    valid_points = pontos[pontos.geometry.apply(lambda x: x is not None and not x.is_empty and x.geom_type == 'Point')]
    if not valid_points.empty:
        center = (valid_points.iloc[0].geometry.y, valid_points.iloc[0].geometry.x)
    else:
         c = rios.unary_union.centroid if not rios.empty else Point(0,0)
         center = (c.y, c.x)

else:
    c = rios.unary_union.centroid if not rios.empty else Point(0,0)
    center = (c.y, c.x)


m = folium.Map(location=center, zoom_start=11, tiles="OpenStreetMap")

# rios
for ls in rios.geometry:
    if isinstance(ls, LineString) and not ls.is_empty:
        folium.PolyLine([(y, x) for x, y in ls.coords], weight=2, opacity=0.7).add_to(m)


# tronco principal
for ls in mainstem.geometry:
    if isinstance(ls, LineString) and not ls.is_empty:
        folium.PolyLine([(y, x) for x, y in ls.coords], weight=5, opacity=0.9).add_to(m)


# pontos
for _, r in pontos.iterrows():
    if r.geometry is not None and not r.geometry.is_empty and r.geometry.geom_type == 'Point':
        folium.CircleMarker(
            location=(r.geometry.y, r.geometry.x),
            radius=5, fill=True, tooltip=str(r[ID_COL])
        ).add_to(m)


# caminhos (vermelho)
for f in paths:
    ls = f["geometry"]
    if isinstance(ls, LineString) and not ls.is_empty:
        folium.PolyLine([(y, x) for x, y in ls.coords], weight=4, opacity=0.95, color="red").add_to(m)


m.save(OUT_HTML)
print("OK:", OUT_CSV, OUT_HTML)

OK: /content/distancia_ate_tronco_principal.csv /content/mapa_dist_ate_tronco.html


In [ ]:
import geopandas as gpd
from shapely.ops import linemerge
from shapely.geometry import Point, LineString, MultiLineString
import pandas as pd
import geopandas as gpd
import fiona
from shapely.geometry import shape, Point, LineString, MultiLineString
from shapely.ops import linemerge, transform as shp_transform
from shapely.strtree import STRtree
import pyproj
import numpy as np
import folium
import networkx as nx

PATH_RIOS   = gpd.read_file('/content/drive/MyDrive/Mestrado/v_hidrografia_grass_2k.gpkg').to_crs(epsg=31983)
PATH_PONTOS = = gpd.read_file('/content/drive/MyDrive/Mestrado/v_pontos_micro_2_BR.gpkg').to_crs(epsg=31983)
CRS_METERS  = 31983   # SIRGAS 2000 / UTM 22S
OUT_CSV     = "/content/distancia_ate_tronco_principal.csv" # Changed output path to /content
OUT_HTML    = "/content/mapa_dist_ate_tronco.html" # Changed output path to /content

# Se existir uma coluna de nome do curso d'água, usamos "Paran" para detectar o tronco
MAINSTEM_NAME_SUBSTR = "paran"
POINT_ID_CAND = ["ponto_de_coleta","id","nome","ponto","label"]

# Buffer distance for finding nearest linestring candidates (in meters)
BUFFER_DIST = 500

# Precisão para "colar" (snap) nós do grafo (tente 3; se rede estiver muito ruidosa, use 2)
SNAP_PREC = 3


# -------------------- HELPERS --------------------
def pick_layer_by_geom(path, wanted=("LineString","MultiLineString")):
    layers = fiona.listlayers(path)
    for lyr in layers:
        with fiona.open(path, layer=lyr) as src:
            gtype = src.schema["geometry"]
            if any(w.lower() in gtype.lower() for w in wanted):
                return lyr
    return layers[0]

def to_linestring(geom):
    """Normaliza MultiLineString -> LineString (maior parte)."""
    if geom is None:
        return None
    if isinstance(geom, LineString):
        return geom
    if isinstance(geom, MultiLineString):
        merged = linemerge(geom)
        if isinstance(merged, LineString):
            return merged
        if isinstance(merged, MultiLineString):
            parts = list(merged.geoms)
            return max(parts, key=lambda g: g.length) if parts else None
        return merged
    return None

def Q(xy, prec=SNAP_PREC):
    """Snap de coordenadas para garantir nós idênticos no grafo."""
    return (round(xy[0], prec), round(xy[1], prec))

# -------------------- LOAD --------------------
lyr_rios = pick_layer_by_geom(PATH_RIOS, ("LineString","MultiLineString"))
lyr_pts  = pick_layer_by_geom(PATH_PONTOS, ("Point","MultiPoint"))

rios   = gpd.read_file(PATH_RIOS,  layer=lyr_rios)
pontos = gpd.read_file(PATH_PONTOS, layer=lyr_pts)

rios   = rios.to_crs(epsg=CRS_METERS)
pontos = pontos.to_crs(epsg=CRS_METERS)

# explode multipoint
pontos = pontos.explode(index_parts=True).reset_index(drop=True)

# ID dos pontos
ID_COL = next((c for c in POINT_ID_CAND if c in pontos.columns), None)
if ID_COL is None:
    ID_COL = "id_ponto"
    pontos[ID_COL] = [f"ponto_{i}" for i in range(len(pontos))]

# normalizar linhas
rios["geom_ls"] = rios.geometry.apply(to_linestring)
rios = rios.dropna(subset=["geom_ls"]).set_geometry("geom_ls", crs=rios.crs)

# -------------------- TRONCO PRINCIPAL --------------------
name_col = next((c for c in ["nm_cursodagua","nome","name","nm_trecho","river_name"] if c in rios.columns), None)
if name_col:
    mask_main = rios[name_col].astype(str).str.lower().str.contains(MAINSTEM_NAME_SUBSTR, na=False)
else:
    # fallback: pega os 2% maiores comprimentos como proxy de tronco
    lens = rios.length
    thr = np.quantile(lens, 0.98)
    mask_main = lens >= thr

mainstem = rios[mask_main]
if mainstem.empty:
    mainstem = rios.sort_values(rios.geometry.length, ascending=False).head(20)

# -------------------- GRAFO DA DRENAGEM --------------------
G = nx.Graph()

def add_ls(ls: LineString):
    coords = [Q(c) for c in ls.coords]  # snap
    for a, b in zip(coords[:-1], coords[1:]):
        seg = LineString([a, b])
        G.add_edge(a, b, weight=seg.length, geometry=seg)

for ls in rios.geometry:
    if isinstance(ls, LineString) and not ls.is_empty:
        add_ls(ls)

# nós-alvo = nós do tronco principal
target_nodes = set()
for ls in mainstem.geometry:
    if isinstance(ls, LineString) and not ls.is_empty:
        c = [Q(xy) for xy in ls.coords]  # snap
        for a, b in zip(c[:-1], c[1:]):
            target_nodes.add(a); target_nodes.add(b)

# -------------------- ÍNDICE ESPACIAL --------------------
sidx = rios.sindex

def nearest_linestring(pt: Point):
    """Finds the nearest linestring to a point using a buffer for candidates."""
    candidates_idx = list(sidx.intersection(pt.buffer(BUFFER_DIST)))
    best_ls, best_d = None, float('inf')

    if not candidates_idx:
        # If no candidates in buffer, try finding the single nearest
        single_nearest_idx = list(sidx.nearest(pt))
        if single_nearest_idx:
             candidates_idx = [single_nearest_idx[0]] # Take only the first result

    for i in candidates_idx:
        ls_candidate = rios.geometry.iloc[i]
        # Ensure ls_candidate is a valid LineString before proceeding
        if isinstance(ls_candidate, LineString) and not ls_candidate.is_empty:
            d = pt.distance(ls_candidate)
            if d < best_d:
                best_d, best_ls = d, ls_candidate

    return best_ls

def shortest_to_mainstem(pt: Point):
    """Conecta o ponto ao SEGMENTO correto da linha mais próxima e roda Dijkstra até o tronco."""
    ls = nearest_linestring(pt)
    if ls is None:
        return None, None, None

    # projeção ao longo da linha
    m_proj = ls.project(pt)
    proj   = ls.interpolate(m_proj)
    tnode  = Q((proj.x, proj.y))  # nó temporário (snap)

    # measures along segments
    coords = list(ls.coords)
    seg_measures = [0.0]
    for a, b in zip(coords[:-1], coords[1:]):
        seg_measures.append(seg_measures[-1] + LineString([a, b]).length)

    # index of segment containing the projection
    i_seg = None
    for i in range(len(coords)-1):
        # Adjust condition to handle floating point comparisons
        if seg_measures[i] <= m_proj + 1e-9 and m_proj - 1e-9 <= seg_measures[i+1]:
             i_seg = i
             break
    if i_seg is None:
         # If projection is exactly at the end of the last segment, consider it part of the last segment
         if abs(m_proj - seg_measures[-1]) < 1e-9 and len(coords) > 1:
             i_seg = len(coords) - 2
         else:
            i_seg = 0  # fallback rare or single segment case


    # adjacent vertices of the segment
    vA = Q(coords[i_seg])
    vB = Q(coords[i_seg+1])

    # lengths along sub-segments (not straight line)
    subA_len = LineString([coords[i_seg], (proj.x, proj.y)]).length
    subB_len = LineString([(proj.x, proj.y), coords[i_seg+1]]).length

    # temporary edges connecting tnode to the correct segment
    # Check if vA and vB are valid nodes in the graph before adding edges
    if vA in G and vB in G:
        G.add_edge(tnode, vA, weight=subA_len, geometry=LineString([tnode, vA]))
        G.add_edge(tnode, vB, weight=subB_len, geometry=LineString([tnode, vB]))
    else:
         # If endpoints are not in the graph, the projection is likely problematic or on a disconnected segment
         if tnode in G: G.remove_node(tnode)
         return None, None, None


    # dijkstra until any target node of the mainstem
    best_len = None
    best_tgt = None
    best_path = None
    try:
        # Check if tnode is in the graph before running dijkstra
        if tnode in G:
            lengths, paths = nx.multi_source_dijkstra(G, sources=[tnode], weight="weight")
            for tgt in target_nodes:
                if tgt in lengths:
                    L = lengths[tgt]
                    if (best_len is None) or (L < best_len):
                        best_len, best_tgt, best_path = L, tgt, paths[tgt]
        else:
             # tnode was not added to the graph, likely due to vA or vB not being in G
             pass

    finally:
        # clean up temporaries
        if tnode in G:
            for nb in list(G.neighbors(tnode)): # Use G.neighbors() for robustness
                 if G.has_edge(tnode, nb): # Double check before removing
                     G.remove_edge(tnode, nb)
            # Re-check degree before removing node in case other edges were added
            if tnode in G and G.degree[tnode] == 0:
                G.remove_node(tnode)

    if best_len is None:
        return None, None, None

    path_ls = LineString(best_path) if best_path and len(best_path) >= 2 else None
    return float(best_len), best_tgt, path_ls


# -------------------- EXECUTAR PARA TODOS OS PONTOS --------------------
rows = []
paths = []

for idx, r in pontos.iterrows():
    geom = r.geometry
    if geom is None or geom.is_empty or geom.geom_type != "Point":
        print(f"Skipping point {r.get(ID_COL, idx)}: invalid geometry")
        continue
    d, tgt, pls = shortest_to_mainstem(geom)
    if d is not None:
        rows.append({"id_ponto": r[ID_COL], "dist_m_ate_tronco": d})
        if pls is not None:
            paths.append({"id_ponto": r[ID_COL], "geometry": pls})
    else:
        print(f"Warning: sem caminho para {r.get(ID_COL, idx)} (rede desconectada ou tronco não encontrado)")

df_out = pd.DataFrame(rows)
if not df_out.empty:
    df_out = df_out.sort_values("dist_m_ate_tronco")
df_out.to_csv(OUT_CSV, index=False)

# -------------------- MAPA FOLIUM --------------------
if len(pontos) > 0:
    valid_points = pontos[pontos.geometry.apply(lambda x: x is not None and not x.is_empty and x.geom_type == "Point")]
    if not valid_points.empty:
        center = (valid_points.iloc[0].geometry.y, valid_points.iloc[0].geometry.x)
    else:
        c = rios.unary_union.centroid if not rios.empty else Point(0,0) # Handle empty rios
        center = (c.y, c.x)
else:
    c = rios.unary_union.centroid if not rios.empty else Point(0,0) # Handle empty rios
    center = (c.y, c.x)

m = folium.Map(location=center, zoom_start=11, tiles="OpenStreetMap")

# rios (azul)
for ls in rios.geometry:
    if isinstance(ls, LineString) and not ls.is_empty:
        folium.PolyLine([(y, x) for x, y in ls.coords], weight=2, opacity=0.7).add_to(m)

# tronco principal (mais grosso)
for ls in mainstem.geometry:
    if isinstance(ls, LineString) and not ls.is_empty:
        folium.PolyLine([(y, x) for x, y in ls.coords], weight=5, opacity=0.9).add_to(m)

# pontos
for _, r in pontos.iterrows():
    g = r.geometry
    if g is not None and not g.is_empty and g.geom_type == "Point":
        folium.CircleMarker(
            location=(g.y, g.x),
            radius=5, fill=True, tooltip=str(r[ID_COL])
        ).add_to(m)

# caminhos (vermelho)
for f in paths:
    ls = f["geometry"]
    if isinstance(ls, LineString) and not ls.is_empty:
        folium.PolyLine([(y, x) for x, y in ls.coords], weight=4, opacity=0.95, color="red").add_to(m)

m.save(OUT_HTML)
print("OK:", OUT_CSV, OUT_HTML)

SyntaxError: invalid syntax (ipython-input-650527391.py, line 16)